In [123]:
import pandas as pd

In [124]:
train_demo = pd.read_csv("traindemographics.csv")
train_perf = pd.read_csv("trainperf.csv")
train_prev = pd.read_csv("trainprevloans.csv")


In [125]:
import pandas as pd

dem = pd.read_csv("traindemographics.csv")
perf = pd.read_csv("trainperf.csv")
prev = pd.read_csv("trainprevloans.csv")

print("DEMOGRAPHICS :", dem.columns.tolist())
print("PERF :", perf.columns.tolist())
print("PREVIOUS LOANS :", prev.columns.tolist())



DEMOGRAPHICS : ['customerid', 'birthdate', 'bank_account_type', 'longitude_gps', 'latitude_gps', 'bank_name_clients', 'bank_branch_clients', 'employment_status_clients', 'level_of_education_clients']
PERF : ['customerid', 'systemloanid', 'loannumber', 'approveddate', 'creationdate', 'loanamount', 'totaldue', 'termdays', 'referredby', 'good_bad_flag']
PREVIOUS LOANS : ['customerid', 'systemloanid', 'loannumber', 'approveddate', 'creationdate', 'loanamount', 'totaldue', 'termdays', 'closeddate', 'referredby', 'firstduedate', 'firstrepaiddate']


In [126]:
df = dem.merge(perf, on="customerid", how="left")
df = df.merge(prev, on="customerid", how="left")


In [127]:
# Agrégation de l'historique des prêts
prev_agg = prev.groupby("customerid").agg({
    "systemloanid": "count",         # nombre de prêts précédents
    "loanamount": ["sum", "mean", "max"],
    "totaldue": ["sum", "mean"],
    "termdays": "mean"
}).reset_index()

# Renommer les colonnes pour plus de clarté
prev_agg.columns = [
    "customerid",
    "prev_count",
    "prev_loanamount_sum",
    "prev_loanamount_mean",
    "prev_loanamount_max",
    "prev_totaldue_sum",
    "prev_totaldue_mean",
    "prev_termdays_mean"
]

prev_agg.head()


,customerid,prev_count,prev_loanamount_sum,prev_loanamount_mean,prev_loanamount_max,prev_totaldue_sum,prev_totaldue_mean,prev_termdays_mean
0,8a1088a0484472eb01484669e3ce4e0b,1,10000.0,10000.000000,10000.0,11500.0,11500.000000,15.000000
1,8a1a1e7e4f707f8b014f797718316cad,4,70000.0,17500.000000,30000.0,89500.0,22375.000000,37.500000
2,8a1a32fc49b632520149c3b8fdf85139,7,90000.0,12857.142857,20000.0,106500.0,15214.285714,19.285714
3,8a1eb5ba49a682300149c3c068b806c7,8,130000.0,16250.000000,30000.0,162400.0,20300.000000,33.750000
4,8a1edbf14734127f0147356fdb1b1eb2,2,20000.0,10000.000000,10000.0,24500.0,12250.000000,22.500000


In [128]:
# 1) Fusion des données clients + performance
df = test_perf.merge(test_demo, on="customerid", how="left")

# 2) Fusion avec historique agrégé
df_final = df.merge(prev_agg, on="customerid", how="left")

# Vérification
print("Taille finale :", df_final.shape)
df_final.head()


Taille finale : (1450, 24)


,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,birthdate,...,bank_branch_clients,employment_status_clients,level_of_education_clients,prev_count,prev_loanamount_sum,prev_loanamount_mean,prev_loanamount_max,prev_totaldue_sum,prev_totaldue_mean,prev_termdays_mean
0,8a858899538ddb8e015390510b321f08,301998974,4,40:48.0,39:35.0,10000,12250.0,30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8a858959537a097401537a4e316e25f7,301963615,10,43:40.0,42:34.0,40000,44000.0,30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,15:11.0,15:04.0,20000,24500.0,30,NaN,1981-09-05 00:00:00.000000,...,NaN,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,00:54.0,00:49.0,30000,34500.0,30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8a858e225a28c713015a30db5c48383d,301959177,4,04:33.0,04:27.0,20000,24500.0,30,NaN,1975-08-25 00:00:00.000000,...,NaN,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
cols_to_drop = ["customerid", "systemloanid", "loannumber", "approveddate",
                "creationdate", "referredby"]

df_final = df_final.drop(columns=[col for col in cols_to_drop if col in df_final.columns])


In [130]:
print(df_final.columns.tolist())

['loanamount', 'totaldue', 'termdays', 'birthdate', 'bank_account_type', 'longitude_gps', 'latitude_gps', 'bank_name_clients', 'bank_branch_clients', 'employment_status_clients', 'level_of_education_clients', 'prev_count', 'prev_loanamount_sum', 'prev_loanamount_mean', 'prev_loanamount_max', 'prev_totaldue_sum', 'prev_totaldue_mean', 'prev_termdays_mean']


In [131]:
df_final.isna().sum()

loanamount                       0
totaldue                         0
termdays                         0
birthdate                     1065
bank_account_type             1065
longitude_gps                 1065
latitude_gps                  1065
bank_name_clients             1065
bank_branch_clients           1446
employment_status_clients     1118
level_of_education_clients    1403
prev_count                    1450
prev_loanamount_sum           1450
prev_loanamount_mean          1450
prev_loanamount_max           1450
prev_totaldue_sum             1450
prev_totaldue_mean            1450
prev_termdays_mean            1450
dtype: int64

In [132]:
df_final

,loanamount,totaldue,termdays,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,prev_count,prev_loanamount_sum,prev_loanamount_mean,prev_loanamount_max,prev_totaldue_sum,prev_totaldue_mean,prev_termdays_mean
0,10000,12250.0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40000,44000.0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000,24500.0,30,1981-09-05 00:00:00.000000,Savings,3.227945,6.586668,UBA,NaN,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30000,34500.0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20000,24500.0,30,1975-08-25 00:00:00.000000,Savings,5.248368,13.059864,UBA,NaN,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,10000,11500.0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,10000,13000.0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447,40000,48000.0,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1448,60000,68100.0,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
df_final.dtypes

loanamount                      int64
totaldue                      float64
termdays                        int64
birthdate                      object
bank_account_type              object
longitude_gps                 float64
latitude_gps                  float64
bank_name_clients              object
bank_branch_clients            object
employment_status_clients      object
level_of_education_clients     object
prev_count                    float64
prev_loanamount_sum           float64
prev_loanamount_mean          float64
prev_loanamount_max           float64
prev_totaldue_sum             float64
prev_totaldue_mean            float64
prev_termdays_mean            float64
dtype: object

In [21]:
pip install scikit-learn


  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl (8.9 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)

   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]


[notice] A new release of pip is available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def clean_and_encode(df, date_cols=['birthdate'], target_col='good_bad_flag'):
    """
    Nettoie et encode un DataFrame pour le machine learning.
    Supprime les colonnes de dates et autres colonnes bloquantes pour SMOTE.
    
    Paramètres :
        df : pandas.DataFrame
        date_cols : liste des colonnes à retirer (colonnes date)
        target_col : nom de la colonne cible à exclure de l'encodage
        
    Retour :
        df_clean : DataFrame nettoyé et One-Hot Encodé, sans colonnes date ni cible
        ohe : objet OneHotEncoder (utile pour appliquer sur test)
    """
    df = df.copy()
    
    # --- Retirer temporairement la colonne cible si elle existe ---
    target_series = None
    if target_col in df.columns:
        target_series = df[target_col]
        df = df.drop(columns=[target_col])
    
    # --- Supprimer les colonnes date ---
    df = df.drop(columns=[col for col in date_cols if col in df.columns])
    
    # --- Colonnes numériques ---
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    df[num_cols] = df[num_cols].fillna(0)
    
    # --- Colonnes objets (strings) ---
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    df[obj_cols] = df[obj_cols].fillna('Unknown')
    
    # --- One-Hot Encoding des colonnes catégorielles ---
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    if obj_cols:
        ohe_encoded = ohe.fit_transform(df[obj_cols])
        ohe_cols = ohe.get_feature_names_out(obj_cols)
        df_ohe = pd.DataFrame(ohe_encoded, columns=ohe_cols, index=df.index)
        df = df.drop(columns=obj_cols)
        df_clean = pd.concat([df, df_ohe], axis=1)
    else:
        df_clean = df.copy()
        ohe = None  # Pas de colonnes catégorielles
    
    # --- Remettre la colonne cible à la fin ---
    if target_series is not None:
        df_clean[target_col] = target_series
    
    return df_clean, ohe


In [135]:
df_clean, ohe = clean_and_encode(df, date_cols=['birthdate'])
df_clean.head()


,systemloanid,loannumber,loanamount,totaldue,termdays,longitude_gps,latitude_gps,customerid_8a28afc7474813a40147639ec637156b,customerid_8a3735d5518aba7301518ac34413010d,customerid_8a76e7d443e6e97c0143ed099d102b1d,...,employment_status_clients_Retired,employment_status_clients_Self-Employed,employment_status_clients_Student,employment_status_clients_Unemployed,employment_status_clients_Unknown,level_of_education_clients_Graduate,level_of_education_clients_Post-Graduate,level_of_education_clients_Primary,level_of_education_clients_Secondary,level_of_education_clients_Unknown
0,301998974,4,10000,12250.0,30,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,301963615,10,40000,44000.0,30,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,301982236,6,20000,24500.0,30,3.227945,6.586668,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,301971730,8,30000,34500.0,30,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,301959177,4,20000,24500.0,30,5.248368,13.059864,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [136]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# -------------------------------
# 1) Séparer la cible
# -------------------------------
y = df_final["good_bad_flag"].map({'Good': 0, 'Bad': 1})  # transformer en 0/1
X = df_final.drop(columns=["good_bad_flag"])


KeyError: 'good_bad_flag'

In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("X_train :", X_train.shape)
print("X_test  :", X_test.shape)
print("y_train :", y_train.shape)
print("y_test  :", y_test.shape)


X_train : (3500, 13690)
X_test  : (876, 13690)
y_train : (3500,)
y_test  : (876,)


In [42]:
pip install imblearn

  Using cached imbalanced_learn-0.14.0-py3-none-any.whl.metadata (8.8 kB)
Using cached imbalanced_learn-0.14.0-py3-none-any.whl (239 kB)

   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalanced-learn]
   ---------------------------------------- 0/2 [imbalance


[notice] A new release of pip is available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print("Avant SMOTE :", X_train.shape, y_train.value_counts())
print("Après SMOTE :", X_resampled.shape, y_resampled.value_counts())


Avant SMOTE : (3500, 13690) good_bad_flag_Bad
0.0    2738
1.0     762
Name: count, dtype: int64
Après SMOTE : (5476, 13690) good_bad_flag_Bad
0.0    2738
1.0    2738
Name: count, dtype: int64


In [62]:
# Remplacer tous les caractères non alphanumériques par un underscore
X_resampled.columns = [c.replace(' ', '_').replace('-', '_').replace('/', '_') for c in X_resampled.columns]

# Ou plus robuste : garder uniquement lettres, chiffres et underscore
import re
X_resampled.columns = [re.sub(r'\W+', '_', c) for c in X_resampled.columns]


In [63]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# Séparer en train et test
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# LightGBM Classifier
lgb_model = lgb.LGBMClassifier(objective='binary', random_state=42, n_jobs=-1)

# Grille de recherche aléatoire
param_dist = {
    'num_leaves': sp_randint(20, 50),
    'learning_rate': sp_uniform(0.01, 0.1),
    'n_estimators': sp_randint(100, 1000),
    'subsample': sp_uniform(0.6, 0.4),
    'colsample_bytree': sp_uniform(0.6, 0.4),
    'min_child_samples': sp_randint(10, 50)
}

# Recherche aléatoire avec 20 combinaisons
random_search = RandomizedSearchCV(
    lgb_model, param_distributions=param_dist,
    n_iter=20, scoring='f1', cv=3, verbose=2, random_state=42
)

# Entraîner
random_search.fit(X_train, y_train)

# Meilleur modèle
best_model = random_search.best_estimator_
print("Meilleurs paramètres :", random_search.best_params_)

# Prédictions
y_pred = best_model.predict(X_test)

# Évaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Number of positive: 1460, number of negative: 1460
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.679037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3068
[LightGBM] [Info] Number of data points in the train set: 2920, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[CV] END colsample_bytree=0.749816047538945, learning_rate=0.10507143064099161, min_child_samples=17, n_estimators=800, num_leaves=40, subsample=0.6624074561769746; total time=  25.0s
[LightGBM] [Info] Number of positive: 1460, number of negative: 1460
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total 

In [64]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Meilleurs paramètres
best_params = {
    'colsample_bytree': 0.6727299868828402,
    'learning_rate': 0.02834045098534338,
    'min_child_samples': 21,
    'n_estimators': 413,
    'num_leaves': 41,
    'subsample': 0.6028265220878869
}

# Création du modèle final
final_model = LGBMClassifier(
    objective='binary',
    random_state=42,
    **best_params
)

# Entraînement
final_model.fit(X_resampled, y_resampled)

# Prédictions
y_pred = final_model.predict(X_test)

# Évaluation
print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


[LightGBM] [Info] Number of positive: 2738, number of negative: 2738
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002071 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4334
[LightGBM] [Info] Number of data points in the train set: 5476, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.9416058394160584

Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94       548
         1.0       0.99      0.90      0.94       548

    accuracy                           0.94      1096
   macro avg       0.95      0.94      0.94      1096
weighted avg       0.95      0.94      0.94      1096


Confusion Matrix:
[[541   7]
 [ 57 491]]


In [66]:
test_demo = pd.read_csv("testdemographics.csv")
test_perf = pd.read_csv("testperf.csv")
test_prev = pd.read_csv("testprevloans.csv")


In [85]:
# --- Agrégation historique prêts TEST ---
prev_agg_test = test_prev.groupby("customerid").agg({
    "systemloanid": "count",
    "loanamount": ["sum", "mean", "max"],
    "totaldue": ["sum", "mean"],
    "termdays": "mean"
}).reset_index()

# Renommer les colonnes
prev_agg_test.columns = [
    "customerid",
    "prev_count",
    "prev_loanamount_sum",
    "prev_loanamount_mean",
    "prev_loanamount_max",
    "prev_totaldue_sum",
    "prev_totaldue_mean",
    "prev_termdays_mean"
]


In [86]:
df = dem.merge(perf, on="customerid", how="left")
df = df.merge(prev_agg, on="customerid", how="left")


In [87]:
# 1) Fusion demographics + perf
df_test = test_demo.merge(test_perf, on="customerid", how="left")

# 2) Fusion avec historique agrégé
df_test_final = df_test.merge(prev_agg_test, on="customerid", how="left")

print("Taille finale :", df_test_final.shape)
df_test_final.head()


Taille finale : (1487, 24)


,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid,...,totaldue,termdays,referredby,prev_count,prev_loanamount_sum,prev_loanamount_mean,prev_loanamount_max,prev_totaldue_sum,prev_totaldue_mean,prev_termdays_mean
0,8a858f305c8dd672015c93b1db645db4,1976-08-28 00:00:00.000000,Savings,5.296628,7.593965,Heritage Bank,NaN,Permanent,NaN,301992953.0,...,11500.0,15.0,NaN,1.0,10000.0,10000.000000,10000.0,13000.0,13000.000000,30.000000
1,8a858f085a477386015a47fb049e49ca,1978-06-23 00:00:00.000000,Savings,3.294513,6.596602,UBA,NaN,Permanent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8a858e6f5cd5e874015cd6f5634c39ad,1984-04-04 00:00:00.000000,Savings,8.501912,7.729364,First Bank,NaN,Permanent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8a858e9d5bfd7037015bfdab79f61305,1983-05-28 00:00:00.000000,Savings,3.318904,6.681595,UBA,NaN,Permanent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8a858fde56eb02280156eb6dafc128ac,1982-03-29 00:00:00.000000,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095.0,...,44000.0,30.0,NaN,13.0,370000.0,28461.538462,40000.0,412800.0,31753.846154,23.076923


In [89]:
cols_to_drop = ["customerid", "systemloanid", "loannumber", "approveddate",
                "creationdate", "referredby"]

df_test_final = df_test_final.drop(columns=[col for col in cols_to_drop if col in df_final.columns])


In [90]:
print(df_test_final.columns.tolist())

['birthdate', 'bank_account_type', 'longitude_gps', 'latitude_gps', 'bank_name_clients', 'bank_branch_clients', 'employment_status_clients', 'level_of_education_clients', 'loanamount', 'totaldue', 'termdays', 'prev_count', 'prev_loanamount_sum', 'prev_loanamount_mean', 'prev_loanamount_max', 'prev_totaldue_sum', 'prev_totaldue_mean', 'prev_termdays_mean']


In [92]:
def clean_and_encode_test(df, ohe, date_cols=['birthdate']):
    df = df.copy()

    # Supprimer dates
    df = df.drop(columns=[col for col in date_cols if col in df.columns])

    # Colonnes numériques
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    df[num_cols] = df[num_cols].fillna(0)

    # Colonnes catégorielles
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    df[obj_cols] = df[obj_cols].fillna('Unknown')

    # Appliquer le OHE du train
    if ohe is not None and obj_cols:
        ohe_encoded = ohe.transform(df[obj_cols])
        ohe_cols = ohe.get_feature_names_out(obj_cols)

        df_ohe = pd.DataFrame(ohe_encoded, columns=ohe_cols, index=df.index)

        df = df.drop(columns=obj_cols)
        df_clean = pd.concat([df, df_ohe], axis=1)
    else:
        df_clean = df.copy()

    return df_clean


In [102]:
# Nettoyage et encodage du test avec le OHE déjà entraîné sur le train
df_test_clean = clean_and_encode_test(df_test_final, ohe)


In [103]:
# On prend les colonnes du train (sans la cible)
train_features = X_train.columns  # X_train = df_train_clean.drop("good_bad_flag", axis=1)

# Aligner le test sur ces colonnes (les colonnes manquantes seront remplies par 0)
df_test_clean = df_test_clean.reindex(columns=train_features, fill_value=0)


In [105]:
# Prédictions classes
y_test_pred = final_model.predict(df_test_clean)

# Prédictions probabilités (optionnel)
y_test_proba =final_model.predict_proba(df_test_clean)[:,1]

# Affichage des premières prédictions
print(y_test_pred[:10])
print(y_test_proba[:10])


[1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.72429283 0.05847506 0.05183637 0.07346214 0.04325185 0.23405383
 0.03168348 0.75056358 0.06378583 0.02374255]


In [118]:
# df_test : DataFrame original avant nettoyage
y_test_pred = final_model.predict(df_test_clean)  # df_test_clean = features nettoyées pour le modèle

submission = pd.DataFrame({
    "customerid": df_test["customerid"],  # <- récupérer depuis l'original
    "Good_Bad_flag": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print("Fichier submission.csv créé avec succès !")


Fichier submission.csv créé avec succès !


In [120]:
df= pd.read_csv("submission.csv")
df

,customerid,Good_Bad_flag
0,8a858f305c8dd672015c93b1db645db4,1.0
1,8a858f085a477386015a47fb049e49ca,0.0
2,8a858e6f5cd5e874015cd6f5634c39ad,0.0
3,8a858e9d5bfd7037015bfdab79f61305,0.0
4,8a858fde56eb02280156eb6dafc128ac,0.0
...,...,...
1482,8a858fc25bafabdc015bb47fda0323e5,0.0
1483,8a858e245c214660015c3397e4db389a,0.0
1484,8a858e8b5bc9e7e6015bcd7374077f0c,0.0
1485,8a858f2e5c699f3a015c77aa22ed7f23,0.0


In [116]:
# features_test : DataFrame de test après get_dummies
# features_train_columns : colonnes du DataFrame utilisé pour l'entraînement

# Ajouter les colonnes manquantes avec 0
for col in features_train_columns:
    if col not in features_test.columns:
        features_test[col] = 0

# Supprimer les colonnes en trop
features_test = features_test[features_train_columns]

# Maintenant on peut prédire
y_test_pred = final_model.predict(features_test)


NameError: name 'features_train_columns' is not defined

In [115]:
y_test_pred = final_model.predict(features)


LightGBMError: The number of features in data (52) is not the same as it was in training data (13690).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [111]:
# Vérifier si 'customerid' existe
features = df_test_final.copy()
if "customerid" in features.columns:
    features = features.drop(columns=["customerid"])

# Prédiction
y_test_pred =final_model.predict(features)

# Créer le DataFrame de soumission
# Utiliser df_test_final si 'customerid' existe sinon df_test original
if "customerid" in df_test_final.columns:
    customer_ids = df_test_final["customerid"]
else:
    customer_ids = df_test["customerid"]  # DataFrame original

submission = pd.DataFrame({
    "customerid": customer_ids,
    "Good_Bad_flag": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print("Fichier submission.csv créé avec succès !")


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: birthdate: object, bank_account_type: object, bank_name_clients: object, bank_branch_clients: object, employment_status_clients: object, level_of_education_clients: object

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def clean_and_encode(df, date_cols=['birthdate']):
    """
    Nettoie et encode un DataFrame pour le machine learning.
    Supprime les colonnes de dates et autres colonnes bloquantes pour SMOTE.
    
    Paramètres :
        df : pandas.DataFrame
        date_cols : liste des colonnes à retirer (colonnes date)
        
    Retour :
        df_clean : DataFrame nettoyé et One-Hot Encodé, sans colonnes date
        ohe : objet OneHotEncoder (utile pour appliquer sur test)
    """
    df = df.copy()
    
    # --- Supprimer les colonnes date ---
    df = df.drop(columns=[col for col in date_cols if col in df.columns])
    
    # --- Colonnes numériques ---
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    df[num_cols] = df[num_cols].fillna(0)
    
    # --- Colonnes objets (strings) ---
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    df[obj_cols] = df[obj_cols].fillna('Unknown')
    
    # --- One-Hot Encoding des colonnes catégorielles ---
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    if obj_cols:
        ohe_encoded = ohe.fit_transform(df[obj_cols])
        ohe_cols = ohe.get_feature_names_out(obj_cols)
        df_ohe = pd.DataFrame(ohe_encoded, columns=ohe_cols, index=df.index)
        df = df.drop(columns=obj_cols)
        df_clean = pd.concat([df, df_ohe], axis=1)
    else:
        df_clean = df.copy()
        ohe = None  # Pas de colonnes catégorielles
    
    return df_clean, ohe


KeyError: "['customerid'] not found in axis"